In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
import os
import PIL
import tensorflow as tf
import tensorflow_datasets as tfds
import glob
import cv2 as cv
from tqdm import tqdm
import time
from PIL import Image

Loading DATA 


In [ ]:
train = r"C:\Users\user\Documents\RAPID-master\datasets\training_data\images"
test = r"C:\Users\user\Documents\RAPID-master\datasets\test_data\images"

trainlb = np.load(r'C:\Users\user\Documents\RAPID-master\datasets\training_data\labels.npy')
testlb = np.load(r'C:\Users\user\Documents\RAPID-master\datasets\test_data\labels.npy')

# Preprocessing the data 

In [ ]:
x_tr = []
y_tr = []
i = 0
def create_training_data():
        i = 0
        for img in tqdm(os.listdir(train)):
                img_array = cv.imread(os.path.join(train,img) ,cv.IMREAD_COLOR)  # convert to array
                new_array = np.array(cv.resize(img_array, (224, 224))) # resize data
                mean = np.mean(new_array, axis=(0, 1)) # mean for data centering
                std = np.std(new_array, axis=(0, 1)) # standard deviation
                normalized_image_array = (new_array - mean) / std # normalize the image
                x_tr.append(normalized_image_array)
                y_tr.append(trainlb[i])
                i =i+1 
create_training_data()

print(len(x_tr))

In [ ]:
x_ts = []
y_ts = []
i = 0
def create_training_data():
        i = 0
        for img in tqdm(os.listdir(test)):
                img_array = cv.imread(os.path.join(test,img) ,cv.IMREAD_COLOR)  # convert to array
                new_array = np.array(cv.resize(img_array, (224, 224)))  # resize data
                mean = np.mean(new_array, axis=(0, 1)) # mean for data centering
                std = np.std(new_array, axis=(0, 1)) # standard deviation
                normalized_image_array = (new_array - mean) / std # normalize the image
                x_ts.append(normalized_image_array) 
                y_ts.append(testlb[i])
                i =i+1 
create_training_data()

print(len(x_ts))

In [ ]:
print(np.shape(x_tr), np.shape((y_tr)))
print(np.shape(x_ts), np.shape((y_ts)))

# DATA VISUALISATION


we took the vecctors and reshaped them into a 20x10 matrix and streteched them to 224x224 in order to simulate the areas with the lowest moisture dissapation rates 

In [ ]:
plt.figure(figsize=(14,10))
for i in range(8):
  
  plt.subplot(4,4,i+1)
  imageI = cv.resize((trainlb[i].reshape(20,10)),(224,224),  interpolation= cv.INTER_LINEAR)
  plt.imshow(imageI, cmap= 'viridis')
  plt.subplots_adjust(hspace=0.5)
  plt.title("Image{}".format(i))
  plt.axis('on')




we visualized a sample of the images  of the aerial view of the vinyard

In [ ]:
plt.figure(figsize=(14,10))

for i in range(8):
  
  plt.subplot(4,4,i+1)
  image= x_tr[i]
  plt.imshow(image, cmap = 'viridis')
  plt.subplots_adjust(hspace=0.5)
  plt.title("Image{}".format(i))
  plt.axis('on')

# simple CNN 

In [ ]:
x_tr = tf.stack(x_tr)  # convert to tensor
y_tr = tf.stack(y_tr) # convert to tensor
x_ts = tf.stack(x_ts) # convert to tensor
y_ts = tf.stack(y_ts) # convert to tensor

our CNN is composed of 2 conculutional layers followed with a max pooling and a droupout layer and then we adapt the output to suit our task at hand

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Set random seed for reproducibility
tf.random.set_seed(42)


# Create the CNN model
model = models.Sequential()
model.add(layers.Conv2D(16, kernel_size=(3, 3), activation='relu', input_shape=(224, 224,3)))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Dropout(0.4))

model.add(layers.Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Dropout(0.4))
#Flattening 
model.add(layers.Flatten())
#output 
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(200))  # Output layer without activation function
#model summary 
model.summary()

# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam',metrics=['mae'])

Early_Stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,

)

# Train the model
batch_size = 32
epochs = 50

hist= model.fit(x_tr, y_tr, batch_size=batch_size , epochs=epochs, validation_data=(x_ts, y_ts),callbacks=[Early_Stopping])

In [ ]:

plt.rcParams.update({'font.size': 7})
plt.plot(hist.history["loss"],c='b',label="Ens. d'apprentissage")
plt.plot(hist.history["val_loss"],c='r',label="Ens. de test")
plt.title("CNN with dropout MSE")
plt.xlabel("Epoch")
plt.ylabel("MSE")
plt.ylim([0, 1])
plt.legend()
plt.show()


In [ ]:
plt.plot(hist.history['mae'])
plt.plot(hist.history['val_mae'])
plt.title("CNN with dropout MAE")
plt.ylabel('MAE')
plt.xlabel('Epoch')
plt.ylim([0, 1])
plt.legend(['Train mae', 'Validation mae'], loc='upper left')
plt.show()

# **Transfer learning**

### InceptionV3


in order to tryout the partial fine tuning we freeze the first 100 layers of the InveptionV3 model


In [ ]:
from keras.applications.inception_v3 import InceptionV3
from keras.models import Sequential
from keras.layers import Dense, GlobalAveragePooling2D
from keras.optimizers import Adam

# Load InceptionV3 model without the top (fully connected) layers
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers[:100]: # Freeze the first 100 layers
    layer.trainable=False

# Create a new model and add InceptionV3 as a base
model = Sequential()
model.add(base_model)

# Add custom layers on top of InceptionV3 for our output
model.add(GlobalAveragePooling2D())
model.add(Dense(200, activation='relu'))
model.add(Dense(200))  # No activation for regression output

#model.summary
model.summary()

# Compile the model
model.compile(optimizer=Adam(lr=0.01), loss='mean_squared_error',metrics=['mae'])

# Early stopping
Early_Stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
)
# Train the model
histIncce =  model.fit(x_tr, y_tr, batch_size=10, epochs=50, validation_data=(x_ts, y_ts),callbacks=[Early_Stopping])

We visualize the MSE curve for InceptionV3


In [ ]:

plt.rcParams.update({'font.size': 7})
plt.plot(histIncce.history["loss"],c='b',label="Ens. d'apprentissage")
plt.plot(histIncce.history["val_loss"],c='r',label="Ens. de test")
plt.title('MSE results for InceptionV3 without freezed layers')
plt.xlabel("Epoch")
plt.ylabel("MSE")
plt.ylim([0, 0.5])
plt.legend()
plt.show()


We visualize the MAE curve for InceptionV3

In [ ]:
plt.plot(histIncce.history['mae'])
plt.plot(histIncce.history['val_mae'])
plt.title('MAE results for InceptionV3 without freezed layers')
plt.ylabel('MAE')
plt.xlabel('Epoch')
plt.ylim([0, 0.5])
plt.legend(['Train mae loss', 'Validation mae loss'], loc='upper left')
plt.show()

### ResNET50

in order to tryout the partial fine tuning we freeze the first 100 layers of the ResNET50 model

In [ ]:
from keras.applications import ResNet50
#from keras.applications import ResNet50
from keras.models import Sequential
from keras.layers import Dense, GlobalAveragePooling2D
from keras.optimizers import Adam

# Load ResNet50 model without the top (fully connected) layers
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers[:100]: # Freeze the first 100 layers
    layer.trainable=False

# Create a new model and add ResNet50 as a base
model = Sequential()
model.add(base_model)

# Add custom layers on top of ResNet50 for our output
model.add(GlobalAveragePooling2D())
model.add(Dense(200, activation='relu'))
model.add(Dense(200))  

model.summary()
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.01), loss='mean_squared_error',metrics=['mae'])

# Early stopping    
Early_Stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
)

# Train the model
histRN50 =  model.fit(x_tr, y_tr, batch_size=10, epochs=50 ,validation_data=(x_ts, y_ts),callbacks=[Early_Stopping])

We visualize the MSE curve for InceptionV3

In [ ]:

plt.rcParams.update({'font.size': 7})
plt.plot(histRN50.history["loss"],c='b',label="Ens. d'apprentissage")
plt.plot(histRN50.history["val_loss"],c='r',label="Ens. de test")
plt.title('MSE results for ResNET50 without freezed layers')
plt.xlabel("Epoch")
plt.ylabel("MSE")
plt.ylim([0, 40])
plt.legend()
plt.show()

We visualize the MAE curve for InceptionV3

In [ ]:
plt.plot(histRN50.history['mae'])
plt.plot(histRN50.history['val_mae'])
plt.title('MAE results for ResNET50 without freezed layers')
plt.ylabel('MAE')
plt.xlabel('Epoch')
plt.ylim([0, 40])
plt.legend(['Train mae', 'Validation mae'], loc='upper left')
plt.show()

### VGG19

in order to tryout the partial fine tuning we freeze the first 15 layers of the VGG19 model

In [ ]:
from keras.applications.vgg19 import VGG19
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.optimizers import Adam

# Load VGG19 model without the top (fully connected layers)
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers[:15]: # Freeze the first 15 layers
    layer.trainable=False

# Create a new model and add VGG19 as a base
model = Sequential()
model.add(base_model)

# Add custom layers on top of VGG19
model.add(Flatten())
model.add(Dense(200, activation='relu'))
model.add(Dense(200))  # No activation for regression output


model.summary()
# Compile the model
model.compile(optimizer=Adam(lr=0.001), loss='mean_squared_error',metrics=['mae'])

# Early stopping
Early_Stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
)


# Train the model
histVGG =  model.fit(x_tr, y_tr, batch_size=10, epochs=50, validation_data=(x_ts, y_ts),callbacks=[Early_Stopping])

We visualize the MSE curve for VGG19

In [ ]:
plt.rcParams.update({'font.size': 7})
plt.plot(histVGG.history["loss"],c='b',label="Ens. d'apprentissage")
plt.plot(histVGG.history["val_loss"],c='r',label="Ens. de test")
plt.title('MSE results for VGG19 without freezed layers')
plt.xlabel("epoch")
plt.ylabel("MSE")
plt.ylim([0, 0.5])
plt.legend()
plt.show()

We visualize the MAE curve for VGG19

In [ ]:
plt.plot(histVGG.history['mae'])
plt.plot(histVGG.history['val_mae'])
plt.title('MAE results for VGG19 without freezed layers')
plt.ylabel('MAE')
plt.xlabel('Epoch')
plt.ylim([0, 0.5])
plt.legend(['Train mae', 'Validation mae'], loc='upper left')
plt.show()

### Xception

in order to tryout the partial fine tuning we freeze the first 10 layers of the Xception model

In [ ]:
from keras.applications.xception import Xception
from keras.models import Sequential
from keras.layers import Dense, GlobalAveragePooling2D
from keras.optimizers import Adam

# Load Xception model without the top (fully connected) layers
base_model = Xception(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers[:10]: # Freeze the first 10 layers
    layer.trainable=False

# Create a new model and add Xception as a base
model = Sequential()
model.add(base_model)

# Add custom layers on top of Xception for our output
model.add(GlobalAveragePooling2D())
model.add(Dense(200, activation='relu'))
model.add(Dense(200))  

model.summary()



# Compile the model
model.compile(optimizer=Adam(lr=0.01), loss='mean_squared_error',metrics=['mae'])
#Early Stopping  
Early_Stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
)

# Train the model
histXce =  model.fit(x_tr, y_tr, batch_size=10, epochs=50, validation_data=(x_ts, y_ts),callbacks=[Early_Stopping])

We visualize the MSE curve for Xception

In [ ]:
plt.rcParams.update({'font.size': 7})
plt.plot(histXce.history["loss"],c='b',label="Ens. d'apprentissage")
plt.plot(histXce.history["val_loss"],c='r',label="Ens. de test")
plt.title('MSE results for Xception without freezed layers')
plt.xlabel("epoch")
plt.ylabel("MSE")
plt.ylim([0, 0.5])
plt.legend()
plt.show()

We visualize the MAE curve for Xception

In [ ]:
plt.plot(histXce.history['mae'])
plt.plot(histXce.history['val_mae'])
plt.title('MAE results for Xception without  freezed layers')
plt.ylabel('MAE')
plt.xlabel('Epoch')
plt.ylim([0, 0.5])
plt.legend(['Train mae', 'Validation mae'], loc='upper left')
plt.show()